In [0]:
"""
Created on Tue Oct 15 00:14:08 2019

@author: anil
"""

import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers
from google.colab import drive
import csv

drive.mount('/content/drive')



Using TensorFlow backend.


cu80
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
'''
Run the cell if you want to run  validation mode

''' 

"""
Data Importing & Pre-processing

"""

olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
train_olid, test_olid = train_test_split(olid_training, test_size=0.20)

X_train_FULL=train_olid[["id","tweet","subtask_a"]] 
Y_train_FULL=train_olid["subtask_a"]
X_test_FULL=test_olid[["id","tweet","subtask_a"]]
Y_test_FULL=np.array(test_olid[["subtask_a"]])

Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_test_FULL]



X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets

X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)


X_test_FULL["tweet_initial"]=filtered_tweets

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [0]:
'''
Run the cell if you want to run  Test mode

''' 

"""
Data Importing & Pre-processing

"""

import csv
import nltk
nltk.download("popular")

from nltk.tokenize import word_tokenize 

olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
X_test_FULL=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/turkish-goldlabels.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"]=filtered_tweets


z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z




z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z




# BERTurk

In [0]:
"""
Implementation details adapted from https://mccormickml.com/2019/07/22/BERT-fine-tuning/ 
"""

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
!pip install pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
!pip install transformers
from transformers import *



In [0]:
"""
Tokenizing / Preprocessing before BERTurk
"""
tweets_train = X_train_FULL.tweet_initial_nontoken.values

tweets_train = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_train]

X=tweets_train
Y=np.array(Y_TRAIN_ENCODED_FULL)
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

tokenized_X = [tokenizer.tokenize(sent) for sent in X]


print ("Tokenize the first sentence:")
print (tokenized_X[0])

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_X]
MAX_LEN=40
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post",value=0)

attention_masks = []

# Create a masks
for seq in input_ids:
  seq_mask = [int(i>0) for i in seq]
  attention_masks.append(seq_mask)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, Y, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

print(train_inputs.shape)
print(validation_inputs.shape)
print(train_labels)
print(validation_labels)

# Tranforming  data into torch tensors which is the required datatype

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training.
batch_size = 32
"""
 Create an iterator of  input with torch DataLoader. According to the documentation, torch dataloader helps save on memory during training since 
 an iterator the entire dataset does not need to be loaded into memory
"""
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

from transformers import BertForSequenceClassification, AdamW, BertConfig

"""
 Load BertForSequenceClassification, the pretrained BERT model with a single 
 linear classification / Pooling layer on top. 

"""
Berturk_model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased", # Use the 12-layer BERTurk model, with an cased vocabulary.
    num_labels = 2, # The number of output labels 2 for binary classification.
    output_attentions = False, # Decides whether the model returns attentions weights.
    output_hidden_states = False, # Decides whether the model returns all hidden-states.
)

#  For run this model on the GPU. see https://pytorch.org/docs/stable/cuda.html
Berturk_model.cuda()

# Get all of the model's parameters as a list of tuples.

params = list(Berturk_model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# Note: AdamW is an optimizer n class from the huggingface library which is stands for Adam with Weight Decay fix (developed version of classical Adam Optimizer)

optimizer = AdamW(Berturk_model.parameters(),
                  lr = 2e-5, # Learning Rate
                  eps = 1e-8 # args.adam_epsilon 
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4


# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

import numpy as np

from sklearn.metrics import f1_score

# Custom Function to calculate the macro f1 score  of  predictions vs gold labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(classification_report(labels_flat,pred_flat))
    return (np.sum(pred_flat == labels_flat) / len(labels_flat),f1_score(labels_flat,pred_flat,average='macro'))

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


Tokenize the first sentence:
['[CLS]', 'en', 'güzel', 'uyuya', '##n', 'insan', 'ödülü', 'je', '##on', 'j', '##ung', '##ko', '##ok', "'", 'a', 'gidiyor', '.', '.', '.', '[SEP]']
(28149, 40)
(3128, 40)
[0 0 0 ... 1 0 0]
[0 0 1 ... 0 1 0]



The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (32000, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

## Traning

In [0]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the specific seed value  to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch 
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
  
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    Berturk_model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # Unpacking the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        Berturk_model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = Berturk_model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,labels=b_labels
                    )
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss = total_loss + loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        #loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(Berturk_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    Berturk_model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy ,eval_f1= 0, 0,0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = Berturk_model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy,tmp_eval_f1 = flat_accuracy(logits, label_ids)
        #print(report)
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        eval_f1 += tmp_eval_f1

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    880.    Elapsed: 0:00:12.
  Batch    80  of    880.    Elapsed: 0:00:24.
  Batch   120  of    880.    Elapsed: 0:00:36.
  Batch   160  of    880.    Elapsed: 0:00:48.
  Batch   200  of    880.    Elapsed: 0:01:00.
  Batch   240  of    880.    Elapsed: 0:01:12.
  Batch   280  of    880.    Elapsed: 0:01:25.
  Batch   320  of    880.    Elapsed: 0:01:37.
  Batch   360  of    880.    Elapsed: 0:01:49.
  Batch   400  of    880.    Elapsed: 0:02:01.
  Batch   440  of    880.    Elapsed: 0:02:13.
  Batch   480  of    880.    Elapsed: 0:02:26.
  Batch   520  of    880.    Elapsed: 0:02:38.
  Batch   560  of    880.    Elapsed: 0:02:50.
  Batch   600  of    880.    Elapsed: 0:03:03.
  Batch   640  of    880.    Elapsed: 0:03:15.
  Batch   680  of    880.    Elapsed: 0:03:27.
  Batch   720  of    880.    Elapsed: 0:03:40.
  Batch   760  of    880.    Elapsed: 0:03:52.
  Batch   800  of    880.    Elapsed: 0:04:04.
  Batch   840  of

In [0]:
#Saving the Pytorch Model
PATH='/content/drive/My Drive/OFFENSEVAL-TURKISH-MODELS/Berturk-NOTDOWNSAMPLED-4epoch-TESTMODE'


In [0]:

torch.save(Berturk_model, PATH)
# Model class must be defined somewhere

In [0]:

BERTURK_model_loaded = torch.load(PATH)
#bert_model.eval()

### Testing

In [0]:

tweets_test = X_test_FULL.tweet_initial_nontoken.values

X=tweets_test

sentences = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_test]
labels = Y_TEST_ENCODED_FULL

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])
MAX_LEN = 50
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [int(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# create test tensors
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
batch_size = 32  
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
BERTURK_model_loaded.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = BERTURK_model_loaded(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')


  # Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []
for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
probs_berturk=flat_predictions
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]


print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))
print(classification_report(flat_true_labels,flat_predictions,digits=3))

prediction_bert_turk=flat_predictions

# Run Ensemble

In [0]:
testresult_df = pd.read_excel("/content/drive/My Drive/OFFENSEVAL-RESULTS/TEST-PROBS-FINAL.xlsx", sheet_name="Sheet1")

In [0]:
testresult_df

,Unnamed: 0,id,tweet,GOLDLABEL,CNN-LSTM,ATTENTION,BERT-TURKISH,cnnlstm-prob,attention-prob,berturk-prob,Ensemble,Ensemble_probs
0,0,41993,@user sayın başkanım bu şekilde devam inşallah👏,NOT,0,0,0,0.033565,0.109724,0.084147,0,0.084147
1,1,23000,"herkes gevşekliği kadar duyar kasıyor,hayat bö...",NOT,0,0,0,0.117133,0.438991,0.144455,0,0.144455
2,2,42478,olgun ilişkisi olan arkadaş size en güzel hedi...,NOT,0,0,0,0.044277,0.173372,0.074089,0,0.074089
3,3,21748,@user @user burada atıp tutacağına o kötü koşu...,OFF,1,1,1,0.779045,0.817078,0.555502,1,0.701038
4,4,13607,@user i̇şte o onur dediğin sende yok sorun o işte,OFF,1,1,0,0.503417,0.503864,0.201785,0,0.322527
...,...,...,...,...,...,...,...,...,...,...,...,...
3523,3523,27484,@user çok iyi oyuncu. serdar’dan çok çok iyi,NOT,0,0,0,0.027831,0.047612,0.063825,0,0.063825
3524,3524,41742,@user bu gün mənim bəbəşimin ad günüdü💜💜💜💜. tə...,NOT,0,0,0,0.016597,0.016342,0.103336,0,0.103336
3525,3525,45705,bu gecelik bu kadar yarın gün içerisinde göz...,NOT,0,0,0,0.016862,0.031759,0.068938,0,0.068938
3526,3526,29225,@user okulu bırak o zaman fbshshsb,NOT,0,0,0,0.048314,0.148616,0.068575,0,0.068575


In [0]:
testresult_df['berturk-prob-test']= probs_berturk_sigmoided


In [0]:
def sigmoid(x): 
  return 1.0/(1 + np.exp(-x))


sigmoid(probs_berturk[0][0])
probs_berturk_sigmoided=[]
for i in range ( len(probs_berturk)):
  prob0=sigmoid(probs_berturk[i][0])
  prob1=sigmoid(probs_berturk[i][1])

  norm_prob0=prob0/(prob0+prob1)
  norm_prob1=prob1/(prob0+prob1)

  if norm_prob0> norm_prob1:
    probs_berturk_sigmoided.append(1-norm_prob0)
  elif(norm_prob1>=norm_prob0):
    probs_berturk_sigmoided.append(norm_prob1)





In [0]:
#ENSEMBLE with probs
probs_predictions=[]
probs_ensemble=[]
for i in range (len(testresult_df)):
  #p_cnn=testresult_df['cnn-prob'][i]
  p_cnnlstm=testresult_df['cnnlstm-prob'][i]
  p_attention=testresult_df['attention-prob'][i]
  p_berturk=testresult_df['berturk-prob-test'][i]

  if ( p_berturk > 0.85 ):
    probs_predictions.append(1)
    probs_ensemble.append(p_berturk)

  elif(p_berturk < 0.15):
    probs_predictions.append(0)
    probs_ensemble.append(p_berturk)
  elif( (0.15 <=p_berturk<=0.30) or (0.7<=p_berturk<=0.85)):
    
    p_ens=0.6*p_berturk + 0.2*p_cnnlstm + 0.2*p_attention
    if p_ens >= 0.5 :
      probs_predictions.append(1)
      probs_ensemble.append(p_ens)

    else:
      probs_predictions.append(0)
      probs_ensemble.append(p_ens)
  elif(0.3< p_berturk <0.7):
    p_ens= 0.4*p_berturk + 0.3*p_cnnlstm + 0.3*p_attention

    if p_ens >= 0.5:
      probs_predictions.append(1)
      probs_ensemble.append(p_ens)
    else:
      probs_predictions.append(0)
      probs_ensemble.append(p_ens)

In [0]:
GOLDLABEL_encoded = [1 if i == "OFF" else 0 for i in testresult_df['GOLDLABEL']]



In [0]:
print(classification_report(GOLDLABEL_encoded,probs_predictions,digits=3))


              precision    recall  f1-score   support

           0      0.913     0.942     0.927      2812
           1      0.740     0.645     0.690       716

    accuracy                          0.882      3528
   macro avg      0.826     0.794     0.808      3528
weighted avg      0.878     0.882     0.879      3528

